In [208]:
import os
import dotenv
import requests
import json
from io import BytesIO
import time

from IPython.display import display, Image, HTML
import openai
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

In [18]:
# Access variables
dotenv.load_dotenv('.env')

stable_diffusion_api_key = os.getenv('STABLE_DIFFUSION_API_KEY')
openai.api_key = os.getenv("OPENAI_API_KEY")

## Table of Contents (outdated):

IMAGE
* [Fine-tuning model](#Fine-tuning-model)
    * [Check training status](#Check-training-status)
* [Generate new image](#Generate-new-image)
* [Images for prompts](#Images-for-prompts)
    * [Prompt 3](#Prompt-3)
 

### Prompt tuning resources:
- [https://stable-diffusion-art.com/prompt-guide/](https://stable-diffusion-art.com/prompt-guide/)
- [https://midlibrary.io/styles/metal-gear-solid](https://midlibrary.io/styles/metal-gear-solid)
- [https://prompthero.com/stable-diffusion-prompt-guide](https://prompthero.com/stable-diffusion-prompt-guide)
- [Example prompts](https://www.fotor.com/blog/stable-diffusion-prompts/)


### Keyword categories:
1. **Subject**: The subject is what you want to see in the image. A common mistake is not writing enough about the subjects.
2. **Medium**: Medium is the material used to make artwork. Some examples are illustration, oil painting, 3D rendering, and photography. Medium has a strong effect because one keyword alone can dramatically change the style.
3. **Style**: The style refers to the artistic style of the image. Examples include impressionist, surrealist, pop art, etc.
4. **Artist**: Artist names are strong modifiers. They allow you to dial in the exact style using a particular artist as a reference. It is also common to use multiple artist names to blend their styles. Now let’s add Stanley Artgerm Lau, a superhero comic artist, and Alphonse Mucha, a portrait painter in the 19th century.
5. **Website**: Niche graphic websites such as Artstation and Deviant Art aggregate many images of distinct genres. Using them in a prompt is a sure way to steer the image toward these styles.
6. **Resolution**: Resolution represents how sharp and detailed the image is. Let’s add keywords highly detailed and sharp focus.
7. **Additional details**: Additional details are sweeteners added to modify an image. We will add sci-fi, stunningly beautiful and dystopian to add some vibe to the image.
8. **Color**: You can control the overall color of the image by adding color keywords. The colors you specified may appear as a tone or in objects.
9. **Lighting**: Any photographer would tell you lighting is a key factor in creating successful images. Lighting keywords can have a huge effect on how the image looks. Let’s add cinematic lighting and dark to the prompt.


Order matters:
content type > description > style > composition
- **Content type**: What type of artwork you want to achieve? Is it a photograph, drawing, sketch, 3D render..?
- **Description**: define the subject, subject attributes, environment/scene. The more descriptive you are with the use of adjectives, the better the output.
- **Style**: we’ve seen the most common ones above, but there are also “sub-categories” – lightning, detail…
- **Composition**: it refers to aspect ratio, camera view and resolution.

In [19]:
message_prompt = """Give me a birthday card message with an astrology prediction 
for the upcoming year for a person born on 3/24. Their zodiac sign is Aries. 
Make it personal and keep it around one paragraph in length. Do not mention their 
birthday in the response. Focus on the astrology prediction for the upcoming year. 
Do not include salutations or complimentary closings."""

In [26]:
message_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a birthday card writer and astrology predictor."},
        {"role": "user", "content": message_prompt},
    ]
)

In [27]:
message_response.choices[0].message.content

'Happy birthday to an adventurous and passionate Aries! As the stars align for you this year, get ready for an exciting journey filled with new beginnings, personal growth, and abundant opportunities. Your courageous nature will be your greatest strength, pushing you to step out of your comfort zone and embrace bold challenges. The universe will fuel your ambition, allowing you to manifest your dreams and achieve remarkable success. However, remember to balance your fiery energy with patience and diplomacy, as it will be key to maintaining harmonious relationships. Trust your instincts, trust the universe, and prepare for a year of extraordinary possibilities.'

In [20]:
df = pd.DataFrame()

# Fine-tuning model
Note: images must be cropped to 512 x 512 pixels

Model Output:
```
{
    "status":"success",
    "messege":"deploying_gpu",
    "data":"it will take upto 45 minutes.",
    "training_id":"2utUQxLpZbNoAJh05TWMvcHDq",
    "model_id":"2utUQxLpZbNoAJh05TWMvcHDq"
}
```

In [13]:
url = "https://stablediffusionapi.com/api/v3/fine_tune"

payload = json.dumps({
  "key": stable_diffusion_api_key,
  "instance_prompt": "photo of adhik person",
  "class_prompt": "photo of person",
  "base_model_id": "portraitplus-diffusion",
  "images": [
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_1.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_2.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_3.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_4.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_5.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_6.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_7.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_8.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_9.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_10.png"
  ],
  "seed": "0",
  "training_type": "men",
  "max_train_steps": "2000",
  "webhook": ""
})

headers = {
  'Content-Type': 'application/json'
}

training_response = requests.request("POST", url, headers=headers, data=payload)
training_response_json = json.loads(response.text)
print(training_response_json)

{"status":"success","messege":"deploying_gpu","data":"it will take upto 45 minutes.","training_id":"2utUQxLpZbNoAJh05TWMvcHDq","model_id":"2utUQxLpZbNoAJh05TWMvcHDq"}


In [43]:
# model_id = response_json["model_id"]
model_id = "2utUQxLpZbNoAJh05TWMvcHDq"
# training_id = response_json["training_id"]
training_id = "2utUQxLpZbNoAJh05TWMvcHDq"

# Check training status

In [9]:
url = f"https://stablediffusionapi.com/api/v3/fine_tune_status/{model_id}"

payload = json.dumps({
 "key": stable_diffusion_api_key
})

headers = {
  'Content-Type': 'application/json'
}

training_status_response = requests.request("POST", url, headers=headers, data=payload)

In [10]:
training_status_response_json = json.loads(training_status_response.text)

In [11]:
print(training_status_response_json["message"])
print(training_status_response_json["eta"])
print(training_status_response_json["model_download_link"])
print(training_status_response_json["test_image"])

Model is ready to use
0 minutes
https://pub-3626123a908346a7a8be8d9295f44e26.r2.dev/trained-models/2utUQxLpZbNoAJh05TWMvcHDq.tar.gz
https://pub-3626123a908346a7a8be8d9295f44e26.r2.dev/generations/2utUQxLpZbNoAJh05TWMvcHDq.png


# Generate new image

In [327]:
negative_prompt = """multiple figures, duplicate characters, twin images, painting, \
cloned individuals, extra fingers, mutated hands, poorly drawn hands, \
poorly drawn face, deformed, ugly, blurry, bad anatomy, bad proportions, extra limbs, \
cloned face, skinny, glitchy, double torso, extra arms, extra hands, mangled fingers, \
missing lips, ugly face, distorted face, extra legs, anime, naked, no shirt"""

model_config = {
  "key": stable_diffusion_api_key,
  "model_id": model_id,
  "prompt": "",
  "negative_prompt": negative_prompt,
  "width": "1024",
  "height": "728",
  "samples": "1",
  "num_inference_steps": "30",
  "safety_checker": "no",
  "enhance_prompt": "yes",
  "seed": None,
  "guidance_scale": 7.5,
  "multi_lingual": "no",
  "panorama": "no",
  "self_attention": "yes",
  "upscale": "yes",
  "embeddings_model": None,
  "lora_model": None,
  "tomesd": "yes",
  "use_karras_sigmas": "yes",
  "vae": None,
  "lora_strength": None,
  "scheduler": "UniPCMultistepScheduler",
  "webhook": None,
  "track_id": None
}

payload = json.dumps(model_config)

In [343]:
headers = {
  'Content-Type': 'application/json'
}

def generate_img(prompt, config_override):
    options = {"prompt": prompt}
    config = json.dumps({**config_override, **options})  # Merge dictionaries using dictionary unpacking
    response = requests.request("POST", "https://stablediffusionapi.com/api/v4/dreambooth", headers=headers, data=config)
    response_json = json.loads(response.text)
    return response_json

def create(prompt, num_runs=4, options={}):
    combined_config = {**model_config, **options}
    image_urls = []

    print(f"Base prompt: {prompt}")
    
    for run in range(num_runs):
        response_json = generate_img(prompt, combined_config)
        
        # Assuming each 'future_link' in 'future_links' is a URL for a different sample image
        image_urls.extend(response_json["future_links"])
    
    return image_urls

def display_images(image_urls):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    def fetch_image(url):
        response = requests.get(url, headers=headers)
        # Return an HTML img element
        return f'<img src="{url}" style="display:inline-block; margin:1px; width:300px;" />'

    images_html = [fetch_image(url) for url in image_urls]
    
    # Join all img elements into a single string of HTML and display it
    display(HTML(''.join(images_html)))


In [191]:
urls = create("adhik, digital art, motorcycle")

Image #1: adhik digital art
Image #2: adhik digital art motorcycle


In [194]:
display_images(urls)

In [231]:
imgs_3 = create("adhik")
imgs_3

Base prompt: adhik
Prompt #2: adhik, sun
Prompt #3: adhik, sun, nature
Prompt #4: adhik, sun, nature, trees


['https://cdn2.stablediffusionapi.com/generations/0-94c8ec07-4263-4ec1-86af-9c01f98b35d5.png',
 'https://cdn2.stablediffusionapi.com/generations/0-2ffedd53-792a-4edd-bbba-369f3ebc907b.png',
 'https://cdn2.stablediffusionapi.com/generations/0-902e7310-d7ff-4a01-90d6-f32c32fc07af.png',
 'https://cdn2.stablediffusionapi.com/generations/0-a46092e1-94dc-4e6f-ac93-1dc4c2fd1c8f.png']

In [236]:
display_images(imgs_3)

Arthur Adams [https://midlibrary.io/styles/arthur-adams](https://midlibrary.io/styles/arthur-adams)

In [237]:
imgs_4 = create("adhik by Arthur Adams")
imgs_4

Base prompt: adhik
Prompt #2: adhik, by Arthur Adams


['https://cdn2.stablediffusionapi.com/generations/0-50850044-2f85-43b7-8fdd-10c6599203a5.png',
 'https://cdn2.stablediffusionapi.com/generations/0-fb5eff68-767c-4e81-b3e7-ae598d7d30d4.png']

In [242]:
display_images(imgs_4)

In [240]:
imgs_5 = create("adhik by Charles Angrand")
imgs_5

Base prompt: adhik
Prompt #2: adhik, by Charles Angrand


['https://cdn2.stablediffusionapi.com/generations/0-5da139fe-a3e7-41b0-a36d-3a19a79f4cd6.png',
 'https://cdn2.stablediffusionapi.com/generations/0-c45054fb-7316-47e2-8637-7912dcf01173.png']

In [243]:
display_images(imgs_5)

In [245]:
imgs_6 = create("adhik, dress, female, funny")
imgs_6

Base prompt: adhik
Prompt #2: adhik, dress
Prompt #3: adhik, dress, female
Prompt #4: adhik, dress, female, funny


['https://cdn2.stablediffusionapi.com/generations/0-b04164d7-c373-4417-8a92-42f0e62c1ecc.png',
 'https://cdn2.stablediffusionapi.com/generations/0-1b86a938-0703-4810-ba9b-61ebdeb40538.png',
 'https://cdn2.stablediffusionapi.com/generations/0-95eb6f47-a0a1-4630-b278-0f88e40ca3a9.png',
 'https://cdn2.stablediffusionapi.com/generations/0-d50495da-4e26-4524-a8b9-f8e7f9caf656.png']

In [248]:
display_images(imgs_6)

In [382]:
imgs_7 = create("adhik, highly detailed, grainy, realistic, unreal engine, octane render, bokeh, vray, houdini render, quixel megascans, depth of field (or dof), arnold render, 8k uhd, raytracing, cgi, lumen reflections, cgsociety, ultra realistic, volumetric fog, overglaze, analog photo, polaroid, 100mm, film photography, dslr, cinema4d, studio quality")
imgs_7

Base prompt: adhik, highly detailed, grainy, realistic, unreal engine, octane render, bokeh, vray, houdini render, quixel megascans, depth of field (or dof), arnold render, 8k uhd, raytracing, cgi, lumen reflections, cgsociety, ultra realistic, volumetric fog, overglaze, analog photo, polaroid, 100mm, film photography, dslr, cinema4d, studio quality


['https://cdn2.stablediffusionapi.com/generations/0-a26568a7-88fb-4f47-9fb4-8ce9ee005ccb.png',
 'https://cdn2.stablediffusionapi.com/generations/0-facb0154-5048-48ac-b656-e3df4951f84d.png',
 'https://cdn2.stablediffusionapi.com/generations/0-d1477b54-0a24-46c5-866a-128a8c7067fe.png',
 'https://cdn2.stablediffusionapi.com/generations/0-92520b27-7a0b-4a01-bb38-6edfb712b064.png']

In [383]:
display_images(imgs_7)

In [379]:
imgs_8 = create("adhik riding a horse")
imgs_8

Base prompt: adhik riding a horse


['https://cdn2.stablediffusionapi.com/generations/0-ad7455e2-3fbf-47e6-a2e7-4592383e91f7.png',
 'https://cdn2.stablediffusionapi.com/generations/0-a681c663-0f8d-41af-a4b9-4415457b1b91.png',
 'https://cdn2.stablediffusionapi.com/generations/0-bd70cfbe-7db5-498a-a7a9-76bc4bb3b30a.png',
 'https://cdn2.stablediffusionapi.com/generations/0-6d28bc53-1c8a-4608-bcd9-15d61c8cbd37.png']

In [381]:
display_images(imgs_8)

In [374]:
imgs_9 = create("Actual 8K photo, Adhik experiencing a moment of triumph atop a mountain under a starry zodiac-themed sky, reflecting the success foretold in their astrology prediction for the year, amidst a surreal, otherworldly landscape where the earthly and cosmic realms blend seamlessly, Impressionistic style with a blend of realism, reminiscent of Stanley Artgerm Lau’s dynamic superhero stances and Alphonse Mucha’s ethereal color palette and decorative elements, Digital photography blended with digital painting on Artstation, highly detailed with sharp focus, stunningly beautiful with a mystical ambiance, colors reflecting the zodiacal elements, cinematic lighting illuminating Adhik and the symbolic zodiac imagery, wide aspect ratio capturing the grandeur of the moment, mid-angle view encapsulating Adhik’s personal journey and the expansive, dreamlike reality unfolding.")
imgs_9

Base prompt: Actual 8K photo, Adhik experiencing a moment of triumph atop a mountain under a starry zodiac-themed sky, reflecting the success foretold in their astrology prediction for the year, amidst a surreal, otherworldly landscape where the earthly and cosmic realms blend seamlessly, Impressionistic style with a blend of realism, reminiscent of Stanley Artgerm Lau’s dynamic superhero stances and Alphonse Mucha’s ethereal color palette and decorative elements, Digital photography blended with digital painting on Artstation, highly detailed with sharp focus, stunningly beautiful with a mystical ambiance, colors reflecting the zodiacal elements, cinematic lighting illuminating Adhik and the symbolic zodiac imagery, wide aspect ratio capturing the grandeur of the moment, mid-angle view encapsulating Adhik’s personal journey and the expansive, dreamlike reality unfolding.


['https://cdn2.stablediffusionapi.com/generations/0-69a34737-eb60-4ebe-8b04-c516bfcb838a.png',
 'https://cdn2.stablediffusionapi.com/generations/0-f79a838d-b042-4883-93f3-2f08f3f33895.png',
 'https://cdn2.stablediffusionapi.com/generations/0-dc7c8e2b-ff44-457a-824b-a7e8a100bb10.png',
 'https://cdn2.stablediffusionapi.com/generations/0-566a403a-b06d-4d45-8f08-e908b19cfbe5.png']

In [384]:
display_images(imgs_9)

In [373]:
# trying to get this to look more like an astrology prediction
imgs_10 = create("Actual 8K photo, Adhik experiencing a moment of triumph atop a mountain under a starry zodiac-themed sky, reflecting the success foretold in their astrology prediction for the year, amidst a surreal, otherworldly landscape where the earthly and cosmic realms blend seamlessly, Impressionistic style with a blend of realism, reminiscent of Stanley Artgerm Lau’s dynamic superhero stances and Alphonse Mucha’s ethereal color palette and decorative elements, Digital photography blended with digital painting on Artstation, highly detailed with sharp focus, stunningly beautiful with a mystical ambiance, colors reflecting the zodiacal elements, cinematic lighting illuminating Adhik and the symbolic zodiac imagery, wide aspect ratio capturing the grandeur of the moment, mid-angle view encapsulating Adhik’s personal journey and the expansive, dreamlike reality unfolding.")
imgs_10

Base prompt: Actual 8K photo, Adhik experiencing a moment of triumph atop a mountain under a starry zodiac-themed sky, reflecting the success foretold in their astrology prediction for the year, amidst a surreal, otherworldly landscape where the earthly and cosmic realms blend seamlessly, Impressionistic style with a blend of realism, reminiscent of Stanley Artgerm Lau’s dynamic superhero stances and Alphonse Mucha’s ethereal color palette and decorative elements, Digital photography blended with digital painting on Artstation, highly detailed with sharp focus, stunningly beautiful with a mystical ambiance, colors reflecting the zodiacal elements, cinematic lighting illuminating Adhik and the symbolic zodiac imagery, wide aspect ratio capturing the grandeur of the moment, mid-angle view encapsulating Adhik’s personal journey and the expansive, dreamlike reality unfolding.


['https://cdn2.stablediffusionapi.com/generations/0-c6455cff-69a2-4454-8d51-b0a711aad2c5.png',
 'https://cdn2.stablediffusionapi.com/generations/0-286baa13-b4a8-4796-85ab-6524bfa269c4.png',
 'https://cdn2.stablediffusionapi.com/generations/0-952c9a6c-3d67-4ec2-94e2-83fa3230ad53.png',
 'https://cdn2.stablediffusionapi.com/generations/0-01dfbbaa-ff3c-4123-8a79-48489bb4cccb.png']

In [376]:
display_images(imgs_10)

### Good
Note: adding "singular figure, one individual" may have solved the double figure problem

In [365]:
# lots of images have two people. trying to remove second figure
imgs_11 = create("Actual 8K photo, singular figure, one individual, Adhik experiencing a moment of triumph atop a mountain under a starry zodiac-themed sky, reflecting the success foretold in their astrology prediction for the year, amidst a surreal, otherworldly landscape where the earthly and cosmic realms blend seamlessly, Impressionistic style with a blend of realism, reminiscent of Stanley Artgerm Lau’s dynamic superhero stances and Alphonse Mucha’s ethereal color palette and decorative elements, Digital photography blended with digital painting on Artstation, highly detailed with sharp focus, stunningly beautiful with a mystical ambiance, colors reflecting the zodiacal elements, cinematic lighting illuminating Adhik and the symbolic zodiac imagery, wide aspect ratio capturing the grandeur of the moment, mid-angle view encapsulating Adhik’s personal journey and the expansive, dreamlike reality unfolding.")
imgs_11

Base prompt: Actual 8K photo, singular figure, one individual, Adhik experiencing a moment of triumph atop a mountain under a starry zodiac-themed sky, reflecting the success foretold in their astrology prediction for the year, amidst a surreal, otherworldly landscape where the earthly and cosmic realms blend seamlessly, Impressionistic style with a blend of realism, reminiscent of Stanley Artgerm Lau’s dynamic superhero stances and Alphonse Mucha’s ethereal color palette and decorative elements, Digital photography blended with digital painting on Artstation, highly detailed with sharp focus, stunningly beautiful with a mystical ambiance, colors reflecting the zodiacal elements, cinematic lighting illuminating Adhik and the symbolic zodiac imagery, wide aspect ratio capturing the grandeur of the moment, mid-angle view encapsulating Adhik’s personal journey and the expansive, dreamlike reality unfolding.


['https://cdn2.stablediffusionapi.com/generations/0-483ffc9c-5b0c-4b34-8a3e-c627e95c105a.png',
 'https://cdn2.stablediffusionapi.com/generations/0-aa036c9e-df0d-47fc-8f34-0d0c5d20aa6a.png',
 'https://cdn2.stablediffusionapi.com/generations/0-41350b66-6456-4844-9805-2f65ede44a04.png',
 'https://cdn2.stablediffusionapi.com/generations/0-f4be3ac9-bf4a-4f8b-bb0b-941a3754188a.png']

In [372]:
display_images(imgs_11)

### Bad

In [364]:
# testing out a funny prompt
imgs_12 = create("Actual 8K photo, whimsical scene, Adhik amidst a playful chaos of floating, oversized colorful donuts in a surreal, candy-colored landscape, ecstatic expression as a donut halo amusingly frames their head, playful and exaggerated cartoonish style, reminiscent of pop art and the humorous spontaneity of caricature, Digital photography blended with digital illustration on Deviant Art, highly detailed with sharp focus, amusingly bizarre and delightfully whimsical, vibrant color palette with a dominance of pastel tones, comedic lighting enhancing the playful atmosphere, wide aspect ratio capturing the humorous unpredictability of the scene, low-angle view adding to the exaggerated, comedic effect.")
imgs_12

Base prompt: Actual 8K photo, whimsical scene, Adhik amidst a playful chaos of floating, oversized colorful donuts in a surreal, candy-colored landscape, ecstatic expression as a donut halo amusingly frames their head, playful and exaggerated cartoonish style, reminiscent of pop art and the humorous spontaneity of caricature, Digital photography blended with digital illustration on Deviant Art, highly detailed with sharp focus, amusingly bizarre and delightfully whimsical, vibrant color palette with a dominance of pastel tones, comedic lighting enhancing the playful atmosphere, wide aspect ratio capturing the humorous unpredictability of the scene, low-angle view adding to the exaggerated, comedic effect.


['https://cdn2.stablediffusionapi.com/generations/0-00b554bf-a87d-4b86-bf96-9d12604f5c8c.png',
 'https://cdn2.stablediffusionapi.com/generations/0-4eefc81d-1b9d-4605-a06d-e4dba9daf30d.png',
 'https://cdn2.stablediffusionapi.com/generations/0-c9ea454a-a522-4bc6-a187-92339cfc915b.png',
 'https://cdn2.stablediffusionapi.com/generations/0-b5ba34b3-990c-403f-bf2f-dcadb543fa84.png']

In [377]:
display_images(imgs_12)

In [353]:
# another funny one. trying to be more raunchy
imgs_13 = create("Actual 8K photo, singular figure, one individual, Adhik in a goofy scenario, caught off guard in a compromising yet hilarious position, surrounded by cheeky, anthropomorphized vegetables engaging in playful banter, a scene straight out of a bawdy, satirical cartoon, reminiscent of the irreverent humor found in adult animation, digital photography infused with exaggerated, caricatured illustration, on Deviant Art, highly detailed with sharp focus, unabashedly risqué yet light-hearted, vibrant and bold color palette, dramatic, spotlight-style lighting highlighting the comical disbelief on Adhik's face, wide aspect ratio capturing the full comedic spectacle, low-angle view exaggerating the playful absurdity of the scenario.")
imgs_13

Base prompt: Actual 8K photo, singular figure, one individual, Adhik in a goofy scenario, caught off guard in a compromising yet hilarious position, surrounded by cheeky, anthropomorphized vegetables engaging in playful banter, a scene straight out of a bawdy, satirical cartoon, reminiscent of the irreverent humor found in adult animation, digital photography infused with exaggerated, caricatured illustration, on Deviant Art, highly detailed with sharp focus, unabashedly risqué yet light-hearted, vibrant and bold color palette, dramatic, spotlight-style lighting highlighting the comical disbelief on Adhik's face, wide aspect ratio capturing the full comedic spectacle, low-angle view exaggerating the playful absurdity of the scenario.


['https://cdn2.stablediffusionapi.com/generations/0-c06706e3-6bc1-47c3-86fd-0a474bbd207c.png',
 'https://cdn2.stablediffusionapi.com/generations/0-12bfb2b7-ec2f-47f3-ba2f-1c4eff3ea1fe.png',
 'https://cdn2.stablediffusionapi.com/generations/0-622498a6-fc87-433d-9cc7-f09fcb39a231.png',
 'https://cdn2.stablediffusionapi.com/generations/0-d62171e2-a0be-414d-9f12-95f41e12d695.png']

In [363]:
display_images(imgs_13)

In [352]:
# make a man dressed like a woman
imgs_14 = create("Actual 8K photo, singular figure, one individual, Adhik donning an extravagant, over-the-top dress complete with large breasts, a whimsical expression of surprise and amusement on his face, amidst a playful, surreal setting, digital photography blended with humorous illustration on Deviant Art, highly detailed with sharp focus, playful and cheekily irreverent, vibrant color palette with bold, contrasting hues, comedic lighting accentuating the playful elements of the scene, wide aspect ratio capturing the hilarious incongruity, low-angle view.")
imgs_14

Base prompt: Actual 8K photo, singular figure, one individual, Adhik donning an extravagant, over-the-top dress complete with large breasts, a whimsical expression of surprise and amusement on his face, amidst a playful, surreal setting, digital photography blended with humorous illustration on Deviant Art, highly detailed with sharp focus, playful and cheekily irreverent, vibrant color palette with bold, contrasting hues, comedic lighting accentuating the playful elements of the scene, wide aspect ratio capturing the hilarious incongruity, low-angle view.


['https://cdn2.stablediffusionapi.com/generations/0-2072f72b-8379-4a71-b015-542881584478.png',
 'https://cdn2.stablediffusionapi.com/generations/0-843c3760-0885-4b7a-994e-021e17e578bb.png',
 'https://cdn2.stablediffusionapi.com/generations/0-9e57ef40-8bd3-4430-9c88-b9118d21187e.png',
 'https://cdn2.stablediffusionapi.com/generations/0-e54f882a-c4a9-4604-b65c-17967baf2273.png']

In [361]:
display_images(imgs_14)

In [346]:
# Person standing on earth
imgs_15 = create("Actual 8K photo, singular figure, one individual, full body, Adhik standing on planet Earth, amidst the vast expanse of space, a cosmos of stars and planets, embodying a visionary bridging earthly realms and cosmic possibilities, echoing a futuristic, sci-fi aesthetic, the grandeur and mystery of the cosmos echoing the boundless imagination seen in works by Stanley Artgerm Lau and the ethereal allure of Alphonse Mucha’s artistry, Digital photography blended with 3D rendering on Artstation, highly detailed with sharp focus, strikingly magnificent with a cosmic allure, colors reflecting the contrasting coolness of space and the warm hues of earthly realms, cinematic lighting accentuating Adhik as a focal point amidst the vast cosmic theater, wide aspect ratio capturing the epic scale of the scene, mid-angle view encapsulating Adhik’s dominion over earthly domain and the boundless, celestial wonder beyond.")
imgs_15

Base prompt: Actual 8K photo, singular figure, one individual, full body, Adhik standing on planet Earth, amidst the vast expanse of space, a cosmos of stars and planets, embodying a visionary bridging earthly realms and cosmic possibilities, echoing a futuristic, sci-fi aesthetic, the grandeur and mystery of the cosmos echoing the boundless imagination seen in works by Stanley Artgerm Lau and the ethereal allure of Alphonse Mucha’s artistry, Digital photography blended with 3D rendering on Artstation, highly detailed with sharp focus, strikingly magnificent with a cosmic allure, colors reflecting the contrasting coolness of space and the warm hues of earthly realms, cinematic lighting accentuating Adhik as a focal point amidst the vast cosmic theater, wide aspect ratio capturing the epic scale of the scene, mid-angle view encapsulating Adhik’s dominion over earthly domain and the boundless, celestial wonder beyond.


['https://cdn2.stablediffusionapi.com/generations/0-f9d551a9-2bbe-45ad-9914-cb2d62ea56c6.png',
 'https://cdn2.stablediffusionapi.com/generations/0-29153e9f-7686-41a2-9c79-a6b5a314cc91.png',
 'https://cdn2.stablediffusionapi.com/generations/0-cad249e2-7ac3-4b48-b426-ce07bfc34c36.png',
 'https://cdn2.stablediffusionapi.com/generations/0-fe27f572-34ca-445b-b6c3-596ee8139459.png']

In [362]:
display_images(imgs_15)

# First try

In [268]:
# PROMPTS

original_prompt = """actual 8K portrait photo of adhik person, \
portrait, happy colors, bright eyes, clear eyes, warm smile, \
smooth soft skin, big dreamy eyes, beautiful intricate colored \
hair, symmetrical, anime wide eyes, soft lighting, detailed face, \
by makoto shinkai, stanley artgerm lau, wlop, rossdraws, concept art, \
digital painting, looking into camera"""

prompt_1 = """A whimsical illustration of Adhik joyfully experiencing \
his zodiac prediction for the year: [insert zodiac prediction here]. \
Captured in a serene setting with soft lighting accentuating his warm smile \
and bright, dreamy eyes. The portrait, reminiscent of a blend between Makoto \
Shinkai and Stanley Artgerm Lau's art styles, showcases a hopeful Adhik, with \
intricate colored hair, gazing optimistically towards the sky, embodying the \
essence of his astrological forecast. The detailed digital painting vividly \
portrays the harmonious colors and emotions of the moment, as Adhik stands amidst \
a gently blooming garden, symbolizing the promising year ahead."""

prompt_2 = """An enchanting depiction of Adhik embarking on a daring adventure \
symbolizing his Aries spirit. The image captures a courageous Adhik stepping out \ 
from the familiar into a realm filled with new beginnings and endless opportunities, \ 
embodying the essence of personal growth. With a determined gaze and a heart full \
of ambition, he stands at the threshold of a vibrant new world that beckons. The soft \
lighting illuminates his path, highlighting the excitement in his big, dreamy eyes \
and the warm, reassuring smile on his face. His intricate colored hair flows gracefully \
in the wind, adding a whimsical touch to this moment of remarkable potential. The art \
style, a blend of Makoto Shinkai’s ethereal scenery and Stanley Artgerm Lau's evocative \
character design, portrays a blend of courage, anticipation, and the promise of a fulfilling \
year ahead. Every element of this digital painting, from Adhik’s expressive stance to the \
expansive horizon that lays ahead, resonates with the adventurous and passionate energy of \
Aries, promising a year of extraordinary possibilities."""

prompt_3 = """actual 8K portrait photo of adhik person, \
action photo, bright eyes, clear eyes, warm smile, determined eyes, \
beautiful intricate colored hair, symmetrical, anime wide eyes, soft lighting, \
detailed face, by Paul Laffoley, Bess Hamiti, Matt Molloy, Amy Giacomelli, \
concept art, digital painting, looking into camera, birthday card"""




In [45]:
url = "https://stablediffusionapi.com/api/v4/dreambooth"

headers = {
  'Content-Type': 'application/json'
}

gen_response = requests.request("POST", url, headers=headers, data=payload)
gen_response_json = json.loads(gen_response.text)
print(gen_response_json)

{'status': 'processing', 'tip': 'Get 20x faster image generation using enterprise plan. Click here : https://stablediffusionapi.com/enterprise', 'tip_1': 'Your image is processing in background, you can get this image using fetch API: here is link for fetch api : https://stablediffusionapi.com/docs/community-models-api-v4/dreamboothfetchqueimg', 'eta': 8.238667458754739, 'messege': 'Try to fetch request after seconds estimated', 'webhook_status': '', 'fetch_result': 'https://stablediffusionapi.com/api/v3/dreambooth/fetch/51707832', 'id': 51707832, 'output': [], 'meta': {'prompt': 'portrait+ style actual 8K portrait photo of adhik person, action photo, bright eyes, clear eyes, warm smile, determined eyes, beautiful intricate colored hair, symmetrical, anime wide eyes, soft lighting, detailed face, by Paul Laffoley, Bess Hamiti, Matt Molloy, Amy Giacomelli, concept art, digital painting, looking into camera, birthday card hyperrealistic, full body, detailed clothing, highly detailed, cin

In [46]:
fetch_url = gen_response_json["fetch_result"]

payload = json.dumps({
  "key": stable_diffusion_api_key,
})

headers = {
  'Content-Type': 'application/json'
}

fetch_response = requests.request("POST", fetch_url, headers=headers, data=payload)
fetch_response_json = json.loads(gen_response.text)
print(fetch_response_json)

{'status': 'processing', 'tip': 'Get 20x faster image generation using enterprise plan. Click here : https://stablediffusionapi.com/enterprise', 'tip_1': 'Your image is processing in background, you can get this image using fetch API: here is link for fetch api : https://stablediffusionapi.com/docs/community-models-api-v4/dreamboothfetchqueimg', 'eta': 8.238667458754739, 'messege': 'Try to fetch request after seconds estimated', 'webhook_status': '', 'fetch_result': 'https://stablediffusionapi.com/api/v3/dreambooth/fetch/51707832', 'id': 51707832, 'output': [], 'meta': {'prompt': 'portrait+ style actual 8K portrait photo of adhik person, action photo, bright eyes, clear eyes, warm smile, determined eyes, beautiful intricate colored hair, symmetrical, anime wide eyes, soft lighting, detailed face, by Paul Laffoley, Bess Hamiti, Matt Molloy, Amy Giacomelli, concept art, digital painting, looking into camera, birthday card hyperrealistic, full body, detailed clothing, highly detailed, cin

{'status': 'processing', 'tip': 'Get 20x faster image generation using enterprise plan. Click here : https://stablediffusionapi.com/enterprise', 'tip_1': 'Your image is processing in background, you can get this image using fetch API: here is link for fetch api : https://stablediffusionapi.com/docs/community-models-api-v4/dreamboothfetchqueimg', 'eta': 21.20322994262903, 'messege': 'Try to fetch request after seconds estimated', 'webhook_status': '', 'fetch_result': 'https://stablediffusionapi.com/api/v3/dreambooth/fetch/51661231', 'id': 51661231, 'output': [], 'meta': {'prompt': "portrait+ style A whimsical illustration of Adhik joyfully experiencing his zodiac prediction for the year: [insert zodiac prediction here]. Captured in a serene setting with soft lighting accentuating his warm smile and bright, dreamy eyes. The portrait, reminiscent of a blend between Makoto Shinkai and Stanley Artgerm Lau's art styles, showcases a hopeful Adhik, with intricate colored hair, gazing optimistically towards the sky, embodying the essence of his astrological forecast. The detailed digital painting vividly portrays the harmonious colors and emotions of the moment, as Adhik stands amidst a gently blooming garden, symbolizing the promising year ahead. hyperrealistic, full body, detailed clothing, highly detailed, cinematic lighting, stunningly beautiful, intricate, sharp focus, f/1. 8, 85mm, (centered image composition), (professionally color graded), ((bright soft diffused light)), volumetric fog, trending on instagram, trending on tumblr, HDR 4K, 8K", 'model_id': '2utUQxLpZbNoAJh05TWMvcHDq', 'negative_prompt': 'painting, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, deformed, ugly, blurry, bad anatomy, bad proportions, extra limbs, cloned face, skinny, glitchy, double torso, extra arms, extra hands, mangled fingers, missing lips, ugly face, distorted face, extra legs, anime(child:1.5), ((((underage)))), ((((child)))), (((kid))), (((preteen))), (teen:1.5) ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, disfigured, deformed, body out of frame, bad anatomy, watermark, signature, cut off, low contrast, underexposed, overexposed, bad art, beginner, amateur, distorted face, blurry, draft, grainy', 'scheduler': 'UniPCMultistepScheduler', 'safety_checker': 'no', 'W': 1400, 'H': 1000, 'guidance_scale': 7.5, 'seed': 4078858537, 'steps': 20, 'n_samples': 1, 'full_url': 'no', 'tomesd': 'yes', 'upscale': 'no', 'multi_lingual': 'no', 'panorama': 'no', 'self_attention': 'no', 'use_karras_sigmas': 'yes', 'embeddings': None, 'vae': None, 'lora': None, 'lora_strength': 1, 'clip_skip': 1, 'temp': 'no', 'base64': 'no', 'file_prefix': 'e651eaaf-1c19-46a6-aa84-60a885688278.png'}, 'future_links': ['https://cdn2.stablediffusionapi.com/generations/0-e651eaaf-1c19-46a6-aa84-60a885688278.png']}

# Images for prompts

### Prompt 3

In [109]:
print(prompt_3)

actual 8K portrait photo of adhik person, action photo, bright eyes, clear eyes, warm smile, big dreamy eyes, beautiful intricate colored hair, symmetrical, anime wide eyes, soft lighting, detailed face, by Paul Laffoley, Bess Hamiti, Matt Molloy, Amy Giacomelli, Metal Gear Solid, concept art, digital painting, looking into camera


In [123]:
#Image(url='https://cdn2.stablediffusionapi.com/generations/0-6412182c-aa83-4df6-82ad-36917ae1fba6.png') 

In [21]:
#Image(url='https://cdn2.stablediffusionapi.com/generations/0-66a336d3-98f2-480e-98ea-734a911c86df.png') 

In [22]:
#Image(url='https://cdn2.stablediffusionapi.com/generations/0-091e0403-b123-47a7-87bc-40c4af1faafd.png') 

In [ ]:
prompt = "adhik"
